In [14]:
import tensorflow as tf # TF2
import numpy as np
from PIL import Image

In [15]:
print(tf.__version__)

1.15.2


In [16]:
model_file = 'mobilenet_v1_1.0_224_quant.tflite'
image_file = 'dog-1.jpg'
label_file = 'labels_mobilenet_quant_v1_224.txt'
input_mean = 127.5
input_std  = 127.5

In [17]:
interpreter = tf.lite.Interpreter(model_path=model_file)

In [18]:
def load_labels(filename):
      with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]

In [19]:
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
img = Image.open(image_file).resize((width, height))

# add N dim
input_data = np.expand_dims(img, axis=0)

if floating_model:
    input_data = (np.float32(input_data) - input_mean) / input_std

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
results = np.squeeze(output_data)

top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)
for i in top_k:
    if floating_model:
      print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
    else:
      print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

0.976471: Labrador retriever
0.023529: golden retriever
0.000000: toilet tissue
0.000000: starfish
0.000000: guinea pig


In [16]:
np.asarray(img).shape

(224, 224, 3)

## Deploying with EZsmdeploy

In [1]:
!pip install ezsmdeploy

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
%%writefile modelscript_tensorflow_lite.py
import tensorflow as tf
import numpy as np
from PIL import Image
import os
import json

#Return loaded model
def load_model(modelpath):
    model_file = os.path.join(modelpath,'mobilenet_v1_1.0_224_quant.tflite')
    model = tf.lite.Interpreter(model_path=model_file)  
    model.allocate_tensors()
    return model

# return prediction based on loaded model (from the step above) and an input payload
def predict(model, payload):
    try:
        img = Image.frombytes('RGB', (224,224), payload, 'raw')# For Multi model endpoints -> [payload[0]['body'].decode()]
        
        #img = np.frombuffer(data, dtype=np.uint8).reshape((224, 224))
        
        input_details = model.get_input_details()
        output_details = model.get_output_details()

        # check the type of the input tensor
        floating_model = input_details[0]['dtype'] == np.float32

        # NxHxWxC, H:1, W:2
        height = input_details[0]['shape'][1]
        width = input_details[0]['shape'][2]
        #img = Image.open(image_file).resize((width, height))

        # add N dim
        input_data = np.expand_dims(img, axis=0)

        if floating_model:
            input_data = (np.float32(input_data) - input_mean) / input_std

        model.set_tensor(input_details[0]['index'], input_data)

        model.invoke()

        output_data = model.get_tensor(output_details[0]['index'])
        results = np.squeeze(output_data)

        out = results.tolist()
    except Exception as e:
        out = str(e)
    return [json.dumps({'output':[out],'tfeager': tf.executing_eagerly()})]

Overwriting modelscript_tensorflow_lite.py


In [23]:
from modelscript_tensorflow_lite import *
model = load_model('./')

In [24]:
import io
input_data = Image.open('dog-1.jpg').resize((224, 224)).tobytes()
predict(model,input_data)

['{"output": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 249, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [2]:
import ezsmdeploy

In [5]:
!docker container stop $(docker container ls -aq) >/dev/null

In [6]:
ez = ezsmdeploy.Deploy(model = 'mobilenet_v1_1.0_224_quant.tflite', 
                  script = 'modelscript_tensorflow_lite.py',
                  requirements = ['numpy','tensorflow==1.15.2','Pillow'], #or pass in the path to requirements.txt
                  instance_type = 'local_gpu', #... if you intend to deploy on GPU with the flask-ngnix stack
                  wait = True)

0:00:00.206762 | compressed model(s)
0:00:00.580290 | uploaded model tarball(s) ; check returned modelpath
0:00:00.581330 | added requirements file
0:00:00.583143 | added source file
0:00:00.584599 | added Dockerfile
0:00:00.586544 | added model_handler and docker utils
0:00:00.586706 | building docker container
0:01:14.761484 | built docker container
0:01:14.874475 | created model(s). Now deploying on local_gpu
∙∙● Attaching to tmpuseyvt5u_algo-1-e3zjl_1
algo-1-e3zjl_1  | Starting the inference server with 32 workers.
∙∙∙ algo-1-e3zjl_1  | [2020-04-24 00:38:40 +0000] [9] [INFO] Starting gunicorn 20.0.4
algo-1-e3zjl_1  | [2020-04-24 00:38:40 +0000] [9] [INFO] Listening at: unix:/tmp/gunicorn.sock (9)
algo-1-e3zjl_1  | [2020-04-24 00:38:40 +0000] [9] [INFO] Using worker: gevent
algo-1-e3zjl_1  | [2020-04-24 00:38:40 +0000] [13] [INFO] Booting worker with pid: 13
●∙∙ algo-1-e3zjl_1  | [2020-04-24 00:38:41 +0000] [14] [INFO] Booting worker with pid: 14
algo-1-e3zjl_1  | [2020-04-24 00:38:

In [9]:
out = ez.predictor.predict(input_data)

algo-1-e3zjl_1  | received input data
algo-1-e3zjl_1  | b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x

algo-1-e3zjl_1  | 172.27.0.1 - - [24/Apr/2020:00:39:39 +0000] "POST /invocations HTTP/1.1" 200 3037 "-" "-"


In [20]:
results = np.asarray(json.loads(out.decode())['output'][0])
results

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)
for i in top_k:
    if floating_model:
      print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
    else:
      print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

0.976471: Labrador retriever
0.023529: golden retriever
0.000000: toilet tissue
0.000000: starfish
0.000000: guinea pig


In [3]:
ez = ezsmdeploy.Deploy(model = 'mobilenet_v1_1.0_224_quant.tflite', 
                  script = 'modelscript_tensorflow_lite.py',
                  requirements = ['numpy','tensorflow==1.15.2','Pillow'], #or pass in the path to requirements.txt
                  instance_type = 'ml.p2.xlarge', #... if you intend to deploy on GPU with the flask-ngnix stack
                  wait = True)

0:00:00.207165 | compressed model(s)
0:00:00.392749 | uploaded model tarball(s) ; check returned modelpath
0:00:00.393578 | added requirements file
0:00:00.396075 | added source file
0:00:00.397276 | added Dockerfile
0:00:00.399348 | added model_handler and docker utils
0:00:00.399420 | building docker container
0:00:34.681818 | built docker container
0:00:34.793763 | created model(s). Now deploying on ml.p2.xlarge
0:06:06.784313 | deployed model
0:06:06.784881 | estimated cost is $1.361 per hour
0:06:06.784996 | Done! ✔ 


In [26]:
out = ez.predictor.predict(input_data)
out

b'{"output": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 249, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
ez.predictor.delete_endpoint()